Nevronska mreza za sentiment analysis

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
 !gdown 19xU3kUyivRWGQzYXnpwlIjQqdfkD56v4

Downloading...
From: https://drive.google.com/uc?id=19xU3kUyivRWGQzYXnpwlIjQqdfkD56v4
To: /content/twitter_data.csv
100% 10.3M/10.3M [00:00<00:00, 67.8MB/s]


In [ ]:
df=pd.read_csv('https://drive.google.com/uc?id=19xU3kUyivRWGQzYXnpwlIjQqdfkD56v4')
df

,id,user,sentiment,tweet
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...


In [ ]:
df=df.drop(['id','user'],axis=1)
df

,sentiment,tweet
0,Positive,im getting on borderlands and i will murder yo...
1,Positive,I am coming to the borders and I will kill you...
2,Positive,im getting on borderlands and i will kill you ...
3,Positive,im coming on borderlands and i will murder you...
4,Positive,im getting on borderlands 2 and i will murder ...
...,...,...
74677,Positive,Just realized that the Windows partition of my...
74678,Positive,Just realized that my Mac window partition is ...
74679,Positive,Just realized the windows partition of my Mac ...
74680,Positive,Just realized between the windows partition of...


In [ ]:
df['sentiment'].value_counts()

sentiment
Negative      22542
Positive      20832
Neutral       18318
Irrelevant    12990
Name: count, dtype: int64

In [ ]:

for column in df.columns:
    if column == 'sentiment':
        df = df[df[column] != 'Irrelevant']


In [ ]:
df['sentiment'].value_counts()

sentiment
Negative    22542
Positive    20832
Neutral     18318
Name: count, dtype: int64

In [ ]:
df.isnull().sum()

sentiment      0
tweet        571
dtype: int64

In [ ]:
 df = df.dropna(axis=0)

In [ ]:
df.isnull().sum()

sentiment    0
tweet        0
dtype: int64

In [ ]:
X = df['tweet']
Y = pd.get_dummies(df['sentiment'])

In [ ]:
 X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

# Keras

In [ ]:
tokenizer = Tokenizer(num_words=64)
tokenizer.fit_on_texts(X_train.values)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train = sequence.pad_sequences(X_train, maxlen=16)
X_test = sequence.pad_sequences(X_test, maxlen=16)

In [ ]:
from keras.models import Sequential
import keras
from keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, LSTM, Flatten
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
model = Sequential()
model.add(Embedding(64, 32, input_length=X_train.shape[1]))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.compile(loss=keras.losses.CategoricalCrossentropy(), optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=[keras.metrics.Precision(), keras.metrics.Recall()])

In [ ]:
 history = model.fit(X_train, Y_train, validation_split=0.2, batch_size=64, epochs=10)

Epoch 1/10
612/612 [==============================] - 19s 23ms/step - loss: 1.0365 - precision: 0.6232 - recall: 0.1406 - val_loss: 0.9953 - val_precision: 0.6500 - val_recall: 0.2120
Epoch 2/10
612/612 [==============================] - 13s 21ms/step - loss: 0.9861 - precision: 0.6348 - recall: 0.2592 - val_loss: 0.9771 - val_precision: 0.6418 - val_recall: 0.2832
Epoch 3/10
612/612 [==============================] - 12s 20ms/step - loss: 0.9673 - precision: 0.6487 - recall: 0.2976 - val_loss: 0.9665 - val_precision: 0.6571 - val_recall: 0.2942
Epoch 4/10
612/612 [==============================] - 12s 20ms/step - loss: 0.9511 - precision: 0.6562 - recall: 0.3200 - val_loss: 0.9652 - val_precision: 0.6406 - val_recall: 0.3198
Epoch 5/10
612/612 [==============================] - 12s 19ms/step - loss: 0.9357 - precision: 0.6649 - recall: 0.3372 - val_loss: 0.9520 - val_precision: 0.6397 - val_recall: 0.3443
Epoch 6/10
612/612 [==============================] - 11s 19ms/step - loss: 0.92

In [ ]:
preds = model.predict(X_test)
print(classification_report(np.argmax(Y_test.values, axis=1), np.argmax(preds, axis=1)))

383/383 [==============================] - 2s 4ms/step
              precision    recall  f1-score   support

           0       0.53      0.70      0.60      4547
           1       0.55      0.51      0.53      3518
           2       0.59      0.44      0.50      4160

    accuracy                           0.55     12225
   macro avg       0.56      0.55      0.55     12225
weighted avg       0.56      0.55      0.55     12225



# PyTorch

In [ ]:
df = pd.read_csv('https://drive.google.com/uc?id=19xU3kUyivRWGQzYXnpwlIjQqdfkD56v4')
df = df.drop(['id', 'user'], axis=1)

df = df[df['sentiment'] != 'Irrelevant']

df = df.dropna(axis=0)

X = df['tweet'].values
Y = pd.get_dummies(df['sentiment']).values


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
!pip install tiktoken
import tiktoken

In [ ]:
tokenizer=tiktoken.get_encoding("gpt2")

In [ ]:
def tokenizacija_i_pad(text, maxlen=16):
  tokenized_texts = [tokenizer.encode(text) for text in text]
  padded_texts = [tokens[:maxlen] + [0] * (maxlen - len(tokens)) for tokens in tokenized_texts]
  return np.array(padded_texts)


In [ ]:
X_train_seq = tokenizacija_i_pad(X_train)
X_test_seq = tokenizacija_i_pad(X_test)

In [ ]:
import torch.nn as nn
import torch

In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
model = TweetModel(vocab_size, embedding_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
class TweetDataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.Y = torch.tensor(Y, dtype=torch.float)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [ ]:
train_dataset = TweetDataset(X_train_seq, Y_train)
test_dataset = TweetDataset(X_test_seq, Y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
class TweetModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim):
        super(TweetModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv1 = nn.Conv1d(embedding_dim, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.lstm = nn.LSTM(32, 100, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(100, output_dim)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.embedding(x).permute(0, 2, 1)
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x, (hn, cn) = self.lstm(x.permute(0, 2, 1))
        x = self.fc(hn[-1])
        return self.softmax(x)


In [ ]:
vocab_size = 50257
embedding_dim = 32
output_dim = 3

model = TweetModel(vocab_size, embedding_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [ ]:
import torch.nn.functional as F

epochs = 10
for epoch in range(epochs):
    model.train()
    for batch_X, batch_Y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, torch.max(batch_Y, 1)[1])
        loss.backward()
        optimizer.step()

    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for batch_X, batch_Y in test_loader:
            outputs = model(batch_X)
            preds = torch.argmax(outputs, dim=1)
            targets = torch.argmax(batch_Y, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    print(f"Epoch {epoch+1}/{epochs}")
    print(classification_report(all_targets, all_preds))


print("Final Evaluation:")
print(classification_report(all_targets, all_preds))

Epoch 1/10
              precision    recall  f1-score   support

           0       0.64      0.54      0.59      4418
           1       0.59      0.53      0.56      3627
           2       0.51      0.64      0.57      4180

    accuracy                           0.57     12225
   macro avg       0.58      0.57      0.57     12225
weighted avg       0.58      0.57      0.57     12225

Epoch 2/10
              precision    recall  f1-score   support

           0       0.67      0.64      0.65      4418
           1       0.74      0.41      0.53      3627
           2       0.54      0.77      0.63      4180

    accuracy                           0.62     12225
   macro avg       0.65      0.61      0.61     12225
weighted avg       0.65      0.62      0.61     12225

Epoch 3/10
              precision    recall  f1-score   support

           0       0.67      0.74      0.70      4418
           1       0.74      0.50      0.60      3627
           2       0.62      0.72      0.6